# Machine Learning Crash Course
## Bornhack 2019

### Import libraries

In [1]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing import image as kp_image

# Keras is only used to load VGG19 model as a high level API to TensorFlow 
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras import backend as K

# pillow is used for loading and saving images
from PIL import Image

# numPy is used for manipulation of array of object i.e Image in our case
import numpy as np

Using TensorFlow backend.


### Config 

In [2]:
# list of layers to be considered for calculation of Content and Style Loss
content_layers = ['block3_conv3']
style_layers   = ['block1_conv1','block2_conv2','block4_conv3']

num_content_layers = len(content_layers)
num_style_layers   = len(style_layers)

# path where the content and style images are located
content_path = 'data/contents/content-eagle.jpg'
style_path   = 'data/styles/style-pattern-1.jpg'

# save the result as
save_name = 'generated.jpg'

# path to where Vgg19 model weight is located 
vgg_weights = "data/vgg_weights/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"

### Loss Functions

##### Content Loss

In [3]:
def get_content_loss(content, target):
    return tf.reduce_mean(tf.square(content - target)) /2

##### Style Loss

In [4]:
def get_style_loss(base_style, gram_target):

    height, width, channels = base_style.get_shape().as_list()
    gram_style = gram_matrix(base_style)

    # Original eqn as a constant to divide i.e 1/(4. * (channels ** 2) * (width * height) ** 2)
    return tf.reduce_mean(tf.square(gram_style - gram_target)) / (channels**2 * width * height) #(4.0 * (channels ** 2) * (width * height) ** 2)

##### Combine Losses

In [5]:
def compute_loss(model, loss_weights, generated_output_activations, 
                 gram_style_features, content_features, 
                 num_content_layers, num_style_layers):

    generated_content_activations = generated_output_activations[:num_content_layers]
    generated_style_activations   = generated_output_activations[num_content_layers:]

    style_weight, content_weight = loss_weights

    style_score = 0
    content_score = 0

    # Accumulate style losses from all layers
    # Here, we equally weight each contribution of each loss layer
    weight_per_style_layer = 1.0 / float(num_style_layers)
    for target_style, comb_style in zip(gram_style_features, generated_style_activations):
        temp = get_style_loss(comb_style[0], target_style)
        style_score += weight_per_style_layer * temp

    # Accumulate content losses from all layers 
    weight_per_content_layer = 1.0 / float(num_content_layers)
    for target_content, comb_content in zip(content_features, generated_content_activations):
        temp = get_content_loss(comb_content[0], target_content)
        content_score += weight_per_content_layer* temp

    # Get total loss
    loss = style_weight*style_score + content_weight*content_score 


    return loss, style_score, content_score

##### Get Features

In [6]:
def get_feature_representations(model, content_path, style_path, num_content_layers):
    """
    Used to pass content and style image through the model.
    """

    # Load our images in 
    content_image = load_img(content_path)
    style_image   = load_img(style_path)

    # batch compute content and style features
    content_outputs = model(content_image)
    style_outputs   = model(style_image)

    # Get the style and content feature representations from our model  
    style_features   = [ style_layer[0]  for style_layer    in style_outputs[num_content_layers:] ]
    content_features = [ content_layer[0] for content_layer in content_outputs[:num_content_layers] ]

    return style_features, content_features

##### Gram Matrix

In [7]:
def gram_matrix(input_tensor):

    # if input tensor is a 3D array of size Nh x Nw X Nc
    # we reshape it to a 2D array of Nc x (Nh*Nw)
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]

    # get gram matrix 
    gram = tf.matmul(a, a, transpose_a=True)

    return gram

## Model 

In [8]:
# Using Keras Load VGG19 model
def get_model(content_layers,style_layers):

    # Load our model. We load pretrained VGG, trained on imagenet data
    vgg19 = VGG19(weights=None, include_top=False)

    # We don't need to (or want to) train any layers of our pre-trained vgg model, so we set it's trainable to false.
    vgg19.trainable = False

    style_model_outputs = [vgg19.get_layer(name).output for name in style_layers]
    content_model_outputs = [vgg19.get_layer(name).output for name in content_layers]

    model_outputs = content_model_outputs + style_model_outputs

    # Build model 
    return Model(inputs = vgg19.input, outputs = model_outputs),  vgg19

## Style Transfer